In [ ]:
pip install hpelm

In [ ]:
pip install tensorflow

In [ ]:
pip install opencv-python

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install tqdm

In [ ]:
pip install sklearn

In [ ]:
pip install plotly

In [ ]:
import os
import tensorflow.keras as keras
os.environ["KERAS_BACKEND"] = 'tensorflow'
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras.backend as K
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Model, load_model
from keras.layers import Dense, Input, Conv2D, MaxPool2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from sklearn.metrics import precision_recall_fscore_support
from tensorflow.python.keras.utils import np_utils
from keras.models import Model, load_model
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from hpelm import ELM
import plotly

In [ ]:
np.random.seed(22)

import plotly.figure_factory as ff

In [ ]:
def load_organic(norm_path):
        norm_files = np.array(os.listdir(norm_path))
        norm_labels = np.array(['organic']*len(norm_files))

        norm_images = []
        for image in tqdm(norm_files):
            image = cv2.imread(os.path.join(norm_path,image))
            image = cv2.resize(image, dsize=(224,224))
            if(image is not None):
                norm_images.append(image)

        norm_images = np.array(norm_images)

        return norm_images, norm_labels

In [ ]:
def load_recylable(pneu_path):
        pneu_files = np.array(os.listdir(pneu_path))
        pneu_labels = np.array(['recyclable']*len(pneu_files))

        pneu_images = []
        for image in tqdm(pneu_files):
            image = cv2.imread(os.path.join(pneu_path,image))
            image = cv2.resize(image, dsize=(224,224))
            if(image is not None):
                 pneu_images.append(image)

        pneu_images = np.array(pneu_images)

        return pneu_images, pneu_labels

In [ ]:
#training data
dataset_em = "Waste_data/DATASET/TRAIN/O"
dataset_oc = "Waste_data/DATASET/TRAIN/R"

In [ ]:
#testing data
dataset_em_test = "Waste_data/DATASET/TEST/O"
dataset_oc_test = "Waste_data/DATASET/TEST/R"

In [ ]:
norm_images, norm_labels = load_organic(dataset_em)

In [ ]:
pneu_images, pneu_labels = load_recyclable(dataset_oc)

In [ ]:
# Put all train images to X_train
X_train = np.append(norm_images, pneu_images, axis=0)

# Put all train labels to y_train
y_train = np.append(norm_labels, pneu_labels)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
fig, axes = plt.subplots(ncols=7, nrows=2, figsize=(16, 4))

indices = np.random.choice(len(X_train), 14)
counter = 0

for i in range(2):
    for j in range(7):
        axes[i,j].set_title(y_train[indices[counter]])
        axes[i,j].imshow(X_train[indices[counter]], cmap='gray')
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        counter += 1
plt.show()

In [ ]:
# Do the exact same thing as what we have done on train data
norm_images_test, norm_labels_test = load_organic(dataset_em_test)
pneu_images_test, pneu_labels_test = load_recyclable(dataset_oc_test)
X_test = np.append(norm_images_test, pneu_images_test, axis=0)
y_test = np.append(norm_labels_test, pneu_labels_test)

In [ ]:
# Save the loaded images to pickle file for future use
 # Use this to save variables
with open('waste_data.pickle', 'wb') as f:
    pickle.dump((X_train, X_test, y_train, y_test), f)

    # Here's how to load it
    # Use this to load variables
with open('waste_data.pickle', 'rb') as f:
    (X_train, X_test, y_train, y_test) = pickle.load(f)

In [ ]:
# Create new axis on all y data
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]

    # Initialize OneHotEncoder object
one_hot_encoder = OneHotEncoder(sparse=False)


    # Convert all labels to one-hot
y_train_one_hot = one_hot_encoder.fit_transform(y_train)
y_test_one_hot = one_hot_encoder.transform(y_test)


    # reshape for conv layer

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 3)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 3)

In [ ]:
# increase the number of data used for training by creating more samples with some sort of randomness on each of them
datagen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1)

In [ ]:
datagen.fit(X_train)
train_gen = datagen.flow(X_train, y_train_one_hot, batch_size=32)

In [ ]:
width = X_train.shape[1]
height = X_train.shape[2]
channel = 3
NUM_CLASS = 2

hidden_size = 120

# Encoding target
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.fit_transform(y_test)

In [ ]:
print(y_train_enc.shape)

In [ ]:
from keras.applications import vgg16
imported_model=tf.keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [ ]:
dnn_model = Sequential()
dnn_model.add(imported_model)
dnn_model.add(Flatten())
dnn_model.add(Dropout(0.2))
dnn_model.add(Dense(256, activation='relu'))
dnn_model.add(Dropout(0.2))
dnn_model.add(Dense(2, activation='softmax'))

In [ ]:
dnn_model.summary()

In [ ]:
dnn_model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])